In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.26" trl peft accelerate bitsandbytes

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

In [ ]:
from unsloth import FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "model_name", # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

In [ ]:
from transformers import TextStreamer, AutoTokenizer, AutoModelForCausalLM
import torch
import openpyxl

# Tokenizer ve modeli yükleyin
#tokenizer = AutoTokenizer.from_pretrained('modelinizin_yolu')
#model = AutoModelForCausalLM.from_pretrained('modelinizin_yolu').to('cuda')

# TextStreamer nesnesini oluşturun
text_streamer = TextStreamer(tokenizer)

# Excel dosyasını yükleyin
wb = openpyxl.load_workbook('NLP Tablo.xlsx')
sheet = wb.active

# Excel dosyasındaki verileri kullanarak modeli çalıştırın
for row in range(2, sheet.max_row):
    # Excel dosyasından instruction ve input alın
    instruction = sheet[f'B{row}'].value
    input_text = sheet[f'C{row}'].value

    # Model için girdileri hazırlayın
    inputs = tokenizer(
        [
            alpaca_prompt.format(instruction, input_text, "")
        ],
        return_tensors="pt"
    ).to("cuda")

    # Modeli çalıştırın ve sonuçları alın
    outputs = model.generate(**inputs, streamer=text_streamer, max_new_tokens=128)

    # Sonuçları decode edin ve Excel dosyasına yazın
    # Sonuçları decode edin
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # İlk olarak, sadece modelin ürettiği çıktıyı (output) ayıklayın
    output_only = result.split(input_text)[-1].strip()

    # Daha sonra, başındaki 'Response:' yazısını silin
    final_output = output_only.replace('Response:', '').strip()

    # Çıktıyı "olumlu", "olumsuz" veya "nötr" olarak etiketleyin
    if "olumlu" in final_output.lower():
        labeled_output = "olumlu"
    elif "olumsuz" in final_output.lower():
        labeled_output = "olumsuz"
    else:
        labeled_output = "nötr"

    # Son olarak, etiketlenmiş çıktıyı Excel dosyasının F sütununa yazdırın
    sheet[f'S{row}'].value = labeled_output

    # Eğer instructionlar biterse döngüden çık
    if not instruction:
        break

# Excel dosyasını kaydedin
wb.save('New NLP Tablo.xlsx')


In [ ]:
# Benzerlik kontrolü
import openpyxl

def kiyasla(excelDosyasi):
    # Excel dosyasını aç
    workbook = openpyxl.load_workbook(excelDosyasi)
    sheet = workbook.active

    # Toplamda kaç tane aynı var sayacı
    ayni_sayaci = 0

    # İlk 251 satıra bak
    for satir in range(1, 252):
        # T ve U sütunlarını al
        kelime_T = sheet.cell(row=satir, column=5).value
        kelime_U = sheet.cell(row=satir, column=4).value

        # Her iki hücre de doluysa ve büyük-küçük harf duyarlılığı olmadan aynıysa
        if kelime_T and kelime_U and kelime_T.lower() == kelime_U.lower():
            ayni_sayaci += 1

    # Toplamda kaç tane aynı olduğunu ekrana yazdır
    print("Toplamda {} satırda T ve U sütunları aynı kelimeleri içeriyor.".format(ayni_sayaci))

# Kodu çalıştır
kiyasla("NLP Tablo.xlsx")
